In [79]:
from google.colab import files
up = files.upload()

Saving en_train_automatically_labeled_output_complete.jsonl to en_train_automatically_labeled_output_complete.jsonl


In [80]:
import json

def load_jsonl_file_to_records(filename):
    records = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                records.append(json.loads(line))

        return pd.DataFrame(records)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None


In [81]:
import pandas as pd
#train_df= load_jsonl_file_to_records('en_train_automatically_labeled_output_complete.json' )
df = load_jsonl_file_to_records('en_train_automatically_labeled_output_complete.jsonl' )

In [ ]:
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 585.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 845.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [82]:
import re
import spacy
import pandas as pd
import nltk
from collections import Counter
from rake_nltk import Rake
from sentence_transformers import SentenceTransformer, util

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Load spaCy model
nlp = spacy.load("en_core_web_lg")

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize RAKE
rake = Rake()

def extract_named_entities(text):
    """Extract named entities using spaCy."""
    doc = nlp(text)
    named_entities = {ent.text for ent in doc.ents}
    return named_entities

def extract_keywords(text):
    """Extract keywords using RAKE."""
    rake.extract_keywords_from_text(text)
    keywords = rake.get_ranked_phrases()
    return keywords

def split_keywords_into_words(keywords):
    """Split keywords into individual words."""
    words = set()
    for keyword in keywords:
        words.update(keyword.split())
    return words

def split_into_clauses(text):
    """Split text into clauses based on syntactic dependencies using spaCy, ensuring proper spacing."""
    doc = nlp(text)
    clauses = []
    for sentence in doc.sents:
        for token in sentence:
            if token.dep_ in {"advcl", "ccomp", "xcomp", "acl", "relcl"} or token == sentence.root:
                clause_text = " ".join([t.text for t in token.subtree])
                clause_text = re.compile(r"\s([,;.:?!])", re.UNICODE).sub(r"\1", clause_text)  # Fix spaces before punctuation
                clauses.append(clause_text)
    return clauses

def find_best_matching_clause(generated_clause, corrected_clauses):
    """Find the best matching clause in corrected_clauses for a generated_clause using Sentence-BERT."""
    generated_embedding = model.encode(generated_clause, convert_to_tensor=True)
    corrected_embeddings = model.encode(corrected_clauses, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(generated_embedding, corrected_embeddings)
    best_match_idx = similarity_scores.argmax().item()
    best_match_clause = corrected_clauses[best_match_idx]
    best_similarity = similarity_scores[0, best_match_idx].item()
    return best_match_clause, best_similarity

def find_hallucinated_clauses(generated_text, corrected_text):
    """Identify hallucinated clauses where all named entities and keywords are hallucinated, or similarity is below threshold."""
    generated_clauses = split_into_clauses(generated_text)
    corrected_clauses = split_into_clauses(corrected_text)

    hallucinated_clauses = []
    clause_similarity = []
    similarity_threshold = 0.55
    for g_clause in generated_clauses:
        c_clause, similarity = find_best_matching_clause(g_clause, corrected_clauses)
        generated_keywords = split_keywords_into_words(extract_keywords(g_clause))
        corrected_keywords = split_keywords_into_words(extract_keywords(c_clause))

        # Check for common keywords
        common_keywords = generated_keywords.intersection(corrected_keywords)
        if len(generated_keywords) == 0:
            raito_common_keywords = 0
        else:
            raito_common_keywords = len(common_keywords) / len(generated_keywords)


        if  (raito_common_keywords <= 0.33) and similarity <= similarity_threshold:
            hallucinated_clauses.append(g_clause)

        # Add similarity score
        clause_similarity.append(similarity)

    # Filter out subsets and keep the largest clause
    filtered_clauses = []
    hallucinated_clauses_sorted = sorted(hallucinated_clauses, key=len, reverse=True)
    for clause in hallucinated_clauses_sorted:
        if not any(clause != other and clause in other for other in hallucinated_clauses_sorted):
            filtered_clauses.append(clause)

    return filtered_clauses, clause_similarity

def update_generated_text(generated_text, hallucinated_clauses):
    """Update the generated text by replacing hallucinated clauses with an empty string and applying strip."""
    for clause in hallucinated_clauses:
        clause = " ".join(clause.split()).replace(" ,", ",").replace(" .", ".")  # Fix spaces before punctuation
        generated_text = generated_text.replace(clause, "")
    return " ".join(generated_text.split()).strip()

def extract_hallucinated_words(generated_text, corrected_text):
    """Extract hallucinated words by comparing generated and corrected texts."""
    generated_keywords = split_keywords_into_words(extract_keywords(generated_text))
    corrected_keywords = split_keywords_into_words(extract_keywords(corrected_text))
    hallucinated_words = generated_keywords - corrected_keywords
    return hallucinated_words

def get_hard_labels(generated_text, hallucinated_words, hallucinated_clauses):
    """Get start and end indices of hallucinated words and clauses in the generated text."""
    hard_labels = []
    for hallucinated_word in hallucinated_words:
        start_idx = generated_text.lower().find(hallucinated_word.lower())
        if start_idx != -1:
            end_idx = start_idx + len(hallucinated_word)
            hard_labels.append([start_idx, end_idx])
    for hallucinated_clause in hallucinated_clauses:
        clause = " ".join(hallucinated_clause.split()).replace(" ,", ",").replace(" .", ".")  # Ensure consistent spacing
        start_idx = generated_text.find(clause)
        if start_idx != -1:
            end_idx = start_idx + len(clause)
            hard_labels.append([start_idx, end_idx])
    return hard_labels

def apply_combined_logic(row):
    """Apply combined logic to handle single clause and multiple clauses."""
    clauses = split_into_clauses(row["model_output_text"])
    if len(clauses) == 1:
        updated_text = row["model_output_text"]
        hallucinated_clauses = []
        clause_similarity = []
    else:
        hallucinated_clauses, clause_similarity = find_hallucinated_clauses(row["model_output_text"], row["corrected_output"])
        updated_text = update_generated_text(row["model_output_text"], hallucinated_clauses)
    return updated_text, hallucinated_clauses, clause_similarity

# Example texts
generated_text = "The UN's Sustainable City initiative defines a city as one that is: Equipped with infrastructure and services to ensure sustainable and equitable access to a range of basic services, such as water, sanitation, and electricity. Additionally, the program aims to colonize the moon for sustainable urban living."
corrected_text = "The UN's Sustainable City initiative defines a city as one that is: Inclusive, safe, resilient, and sustainable, providing access to safe and affordable housing, sustainable transport systems, and green public spaces;"

# Example of DataFrame analysis
data = {
    "model_output_text": [
        generated_text,
        "Petra van Stoveren won a silver medal in the 2008 Summer Olympics in Beijing, China."
    ],
    "corrected_output": [
        corrected_text,
        "Petra van Staveren won a gold medal in the 1984 Summer Olympics in Los Angeles."
    ]
}

#df = pd.DataFrame(data)

# Apply combined logic to update text and find hallucinated clauses
df[["updated_generated_text", "hallucinated_clauses", "clause_similarity"]] = df.apply(
    lambda row: apply_combined_logic(row), axis=1, result_type="expand"
)

# Apply the functions to the DataFrame
df["hallucinated_words"] = df.apply(lambda row: extract_hallucinated_words(row["updated_generated_text"], row["corrected_output"]), axis=1)
df["hard_labels"] = df.apply(lambda row: get_hard_labels(row["model_output_text"], row["hallucinated_words"], row["hallucinated_clauses"]), axis=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [83]:
df.to_json('en_train_new1.jsonl', orient="records", lines=True)
files.download('en_train_new1.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>